<a href="https://colab.research.google.com/github/abhilb/mycolab/blob/main/BCCD_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the BCCD Dataset

In [ ]:
!curl -L "https://public.roboflow.com/ds/3rjLFcVl91?key=8ORYFTdPjq" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from pathlib import Path
import torch
import torchvision

In [ ]:
print(torch.__version__)
print(torchvision.__version__)

In [ ]:
!pip install pytorch_lightning

In [ ]:
import pytorch_lightning as pl

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
import torchvision.transforms as T
import json
from typing import NamedTuple
from collections import defaultdict

In [ ]:
class BBox(NamedTuple):
  xmin: float
  ymin: float
  xmax: float
  ymax: float
  
  @staticmethod
  def from_xywh(*xywh):
    xmin, ymin, wd, ht = xywh
    xmax = xmin + wd
    ymax = ymin + ht
    return BBox(xmin, ymin, xmax, ymax)

In [ ]:
class Annotation(NamedTuple):
  category: int
  bbox: BBox

In [ ]:
class BCCDDataset(torch.utils.data.Dataset):
  def __init__(self, data_path: Path):
    self.data_path = data_path
    annot_file = data_path / "_annotations.coco.json"
    self.dataset = defaultdict(list)
    self.img_id_map = defaultdict(str)
    with open(annot_file) as f:
      annotation = json.load(f)
      for img_annot in annotation['images']:
        id = img_annot['id']
        img_fname = img_annot['file_name']
        self.img_id_map[id] = img_fname
      for box_annot in annotation['annotations']:
        annot = Annotation(box_annot['category_id'], BBox.from_xywh(*box_annot['bbox']))
        idx = box_annot['image_id']
        self.dataset[id].append(annot)
  def __len__(self) -> int:
    return len(self.dataset.keys())

  def __getitem__(self, idx: int):
    img_ids = list(self.dataset.keys())
    img_id = img_ids[idx]
    img_fname = self.img_id_map[img_id]
    annots = self.dataset[img_id]
    img = read_image(str(self.data_path / img_fname))
    img = T.Resize((320, 320))(img)
    img = img.type(torch.float32) / 255.0

    boxes = []
    labels = []
    for annot in annots:
      labels.append(annot.category)
      boxes.append(
          [
           annot.bbox.xmin * 320 / 416,
           annot.bbox.ymin * 320 / 416,
           annot.bbox.xmax * 320 / 416,
           annot.bbox.ymax * 320 / 416
          ]
      )
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    labels = torch.as_tensor(labels, dtype=torch.int64)
    target = {}
    target['boxes'] = boxes
    target['labels'] = labels

    return img, target

In [ ]:
train_dataset = BCCDDataset(Path.cwd() / 'train')

In [ ]:
for img, target in train_dataset:
  print(img.shape)
  print(target)
  break

In [ ]:
dict(train_dataset.dataset).keys()